In [34]:
from comet_ml import API
import comet_ml
import io
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import defaultdict
import time


from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from colour import Color



In [35]:
comet_api = API(api_key='abUSnAytqEzSzLOxNLP1ohibs')
comet_api.get()

['ketrint', 'schattengenie']

In [36]:
def get_parameter_by_key(exp, key):
    parameters = exp.get_parameters_summary()
    for parameter in parameters:
        if parameter['name'] == key:
            return parameter['valueCurrent']


In [37]:
def get_parameter_by_key(exp, key):
    parameters = exp.get_parameters_summary()
    for parameter in parameters:
        if parameter['name'] == key:
            return parameter['valueCurrent']
    return None

def new_to_old_metric(exp, key):
    metric = exp.get_metrics(key)
    vals = [float(m['metricValue']) for m in metric]
    return vals

def stack_lists(data, n=1000):
    new_data = []
    for d in data:
        if len(d) > n:
            new_data.append(d[:n])
        elif len(d) < n:
            new_data.append(
                np.concatenate([d, d[-1].repeat(n - len(d))])
            )
    return np.vstack(new_data).T

def preprocess_gp(vals):
    return np.minimum.accumulate(vals)

def add_zero_point(vals, point):
    return np.array([point] + vals.tolist())

In [38]:
experiments_names = ['clust-val-final']

In [39]:
experiments = []
for i in range(len(experiments_names)):
    experiments.append(comet_api.get(workspace='ketrint', project_name=experiments_names[i]))

In [40]:
experiments = [item for sublist in experiments for item in sublist]

In [41]:
from collections import defaultdict

good_showers = defaultdict(list)
stuck_showers = defaultdict(list)
broken_showers = defaultdict(list)
lost_showers = defaultdict(list)
x = defaultdict(list)
y = defaultdict(list)
z = defaultdict(list)
tx = defaultdict(list)
ty = defaultdict(list)

def get_data(key):
    good_showers[key].append(new_to_old_metric(exp, "Good showers")[0])
    stuck_showers[key].append(new_to_old_metric(exp, "Stuck showers"))
    broken_showers[key].append(new_to_old_metric(exp, "Broken showers"))
    lost_showers[key].append(new_to_old_metric(exp, "Lost showers"))
    x[key].append(new_to_old_metric(exp, "MAE for x"))
    y[key].append(new_to_old_metric(exp,  "MAE for y"))
    z[key].append(new_to_old_metric(exp, "MAE for z"))
    tx[key].append(new_to_old_metric(exp, "MAE for tx"))
    ty[key].append(new_to_old_metric(exp, "MAE for ty"))


In [42]:
for exp in experiments:
    if exp.get_command()[2] == './data_new/data/rand_preprocessed.pt':
        if exp.get_command()[-5] == 'True':
            key = 'HDBSCAN'
            get_data(key)
        else: 
            if exp.get_command()[12] == '0':  
                key = 'Pure_emulsion'
                get_data(key)
            elif exp.get_command()[12] == '4':
                key = 'Equal Mix'
                get_data(key)
            elif exp.get_command()[12] == '3':       
                key = 'Mix Emulsion'
                get_data(key)



In [43]:
dict_list = [good_showers, stuck_showers, 
                   broken_showers, lost_showers, 
                   x, y, z, tx, ty]

In [44]:
dict_names = ['good_showers', 'stuck_showers', 
                   'broken_showers', 'lost_showers', 
                   'MAE for x', 'MAE for y', 'MAE for z', 'MAE for tx', 'MAE for ty']

In [45]:
for dictionary in dict_list:
    for key in dictionary: 
        dictionary[key] = [np.round(np.mean(dictionary[key]),3), 
                           np.round(np.std(dictionary[key]), 3)]

In [46]:
df = pd.DataFrame.from_dict(dict_list)

In [47]:
df = df.assign(Metric = dict_names)

In [48]:
df = df.set_index('Metric') 

In [49]:
df

,Mix Emulsion,HDBSCAN,Pure_emulsion,Equal Mix
Metric,,,,
good_showers,"[0.868, 0.021]","[0.706, 0.029]","[0.829, 0.026]","[0.581, 0.392]"
stuck_showers,"[0.088, 0.03]","[0.137, 0.033]","[0.124, 0.028]","[0.394, 0.399]"
broken_showers,"[0.04, 0.009]","[0.157, 0.045]","[0.04, 0.004]","[0.02, 0.01]"
lost_showers,"[0.004, 0.001]","[0.0, 0.0]","[0.008, 0.003]","[0.005, 0.002]"
MAE for x,"[0.308, 0.002]","[0.312, 0.001]","[0.308, 0.002]","[0.309, 0.003]"
MAE for y,"[0.244, 0.001]","[0.246, 0.002]","[0.244, 0.001]","[0.248, 0.005]"
MAE for z,"[0.207, 0.001]","[0.205, 0.002]","[0.205, 0.002]","[0.195, 0.015]"
MAE for tx,"[0.009, 0.0]","[0.009, 0.0]","[0.008, 0.0]","[0.008, 0.0]"
MAE for ty,"[0.009, 0.0]","[0.009, 0.0]","[0.008, 0.0]","[0.009, 0.001]"


# Cross-validation performance of networks trained on different train datas

In [19]:
experiments_names = ['clust-cross', 'test-hypo']

experiments = []
for i in range(len(experiments_names)):
    experiments.append(comet_api.get(workspace='ketrint', project_name=experiments_names[i]))
    
experiments = [item for sublist in experiments for item in sublist]

In [20]:
len(experiments)

24

In [25]:
experiments[23].get_command()

['clustering.py',
 '--datafile',
 './data_new/data/rand_preprocessed.pt',
 '--start',
 '67',
 '--end',
 '100',
 '--clusters_file',
 './data_new/clusters_rand.pt',
 '--num_layers_emulsion',
 '5',
 '--num_layers_edge_conv',
 '3',
 '--hidden_dim',
 '32',
 '--output_dim',
 '32',
 '--graph_embedder',
 'GraphNN_KNN_v2',
 '--edge_classifier',
 'EdgeClassifier_v1',
 '--project_name',
 'test_hypo',
 '--workspace',
 'ketrint',
 '--z_file',
 './data_new/z.npy',
 '--graph_embedder_weights',
 'graph_embedder_50_preprocessed_train_d8efa5076b9345649794c7f494a1fe42.pt',
 '--edge_classifier_weights',
 'edge_classifier_50_preprocessed_train_d8efa5076b9345649794c7f494a1fe42.pt',
 '--cl_size',
 '30',
 '--min_cl',
 '30',
 '--threshold',
 '0.6',
 '--min_samples_core',
 '2',
 '--vanilla_hdbscan',
 'False',
 '--energy_file',
 './data_new/e.npy',
 '--energy_true_file',
 './data_new/E.npy']

In [26]:
for exp in experiments: 
    if exp.get_command()[2] == './data_new/data/rand_preprocessed.pt':
        if exp.get_command()[-15].split('_')[2] == '200':
            key = '200_rand'
            get_data(key)
        if exp.get_command()[-15].split('_')[2] == '50':
            key = '50_rand'
            get_data(key)           
    else:
        if exp.get_command()[-19].split('_')[2] == 'rand' and exp.get_command()[-13] == '50':  
            key = 'rand_50'
            get_data(key)
        elif exp.get_command()[-19].split('_')[2] == 'rand' and exp.get_command()[-13] == '200':
            key = 'rand_200'
            get_data(key)
        elif exp.get_command()[-19].split('_')[2] == '200' and exp.get_command()[-13] == '200':      
            key = '200_200'
            get_data(key)
        elif exp.get_command()[-19].split('_')[2] == '200' and exp.get_command()[-13] == '50':      
            key = '200_50'
            get_data(key)
        elif exp.get_command()[-19].split('_')[2] == '50' and exp.get_command()[-13] == '50':      
            key = '50_50'
            get_data(key)
        elif exp.get_command()[-19].split('_')[2] == '50' and exp.get_command()[-13] == '200':      
            key = '50_200'
            get_data(key)
        elif exp.get_command()[-19].split('_')[2] == '50' and exp.get_command()[-13] == '200':      
            key = '50_200'
            get_data(key)

In [27]:
dict_list = [good_showers, stuck_showers, 
                   broken_showers, lost_showers, 
                   x, y, z, tx, ty]

In [28]:
dict_names = ['good_showers', 'stuck_showers', 
                   'broken_showers', 'lost_showers', 
                   'MAE for x', 'MAE for y', 'MAE for z', 'MAE for tx', 'MAE for ty']

In [29]:
for dictionary in dict_list:
    for key in dictionary: 
        dictionary[key] = [np.round(np.mean(dictionary[key])*100,2), 
                           np.round(np.std(dictionary[key])*100,2)]

In [30]:
df = pd.DataFrame.from_dict(dict_list)

In [31]:
df = df.assign(Metric = dict_names)

In [32]:
df = df.set_index('Metric') 

In [33]:
df

,rand_50,rand_200,50_50,200_50,50_200,200_200,200_rand,50_rand
Metric,,,,,,,,
good_showers,"[91.52, 0.06]","[83.26, 2.17]","[86.0, 2.53]","[88.99, 0.68]","[75.24, 0.89]","[78.48, 0.25]","[82.2, 1.91]","[82.1, 2.46]"
stuck_showers,"[5.53, 0.23]","[13.08, 2.42]","[12.1, 2.25]","[9.05, 0.63]","[21.93, 0.89]","[18.63, 0.13]","[15.03, 2.21]","[14.86, 2.75]"
broken_showers,"[2.0, 0.32]","[2.19, 0.32]","[1.26, 0.19]","[1.25, 0.06]","[1.43, 0.15]","[1.49, 0.11]","[2.49, 0.37]","[2.68, 0.3]"
lost_showers,"[0.95, 0.61]","[1.47, 0.13]","[0.64, 0.16]","[0.71, 0.04]","[1.4, 0.24]","[1.39, 0.08]","[0.28, 0.07]","[0.36, 0.08]"
MAE for x,"[30.69, 0.3]","[30.71, 0.38]","[30.69, 0.18]","[30.77, 0.23]","[30.79, 0.44]","[30.75, 0.38]","[30.89, 0.11]","[30.86, 0.14]"
MAE for y,"[24.39, 0.14]","[24.34, 0.16]","[24.4, 0.15]","[24.31, 0.15]","[24.38, 0.17]","[24.31, 0.24]","[24.33, 0.14]","[24.43, 0.17]"
MAE for z,"[20.64, 0.16]","[20.61, 0.16]","[20.53, 0.24]","[20.6, 0.16]","[20.52, 0.22]","[20.57, 0.18]","[20.67, 0.22]","[20.64, 0.25]"
MAE for tx,"[0.87, 0.02]","[0.86, 0.03]","[0.88, 0.03]","[0.88, 0.02]","[0.88, 0.03]","[0.88, 0.03]","[0.87, 0.0]","[0.87, 0.04]"
MAE for ty,"[0.86, 0.0]","[0.85, 0.01]","[0.87, 0.01]","[0.88, 0.01]","[0.86, 0.03]","[0.86, 0.03]","[0.88, 0.01]","[0.87, 0.02]"
